# **Bringing in all data**

## Bring in GRID table
*   Download from here: https://www.grid.ac/downloads
*   Select the excress information needed, merge in excel (VLOOKUP) and import here.




In [ ]:
########################################################################
 ## Bring in pre-merged GRID data: 
   # Used latest GRID version: 6/29/2020
   # Columns include: ID, Name, City, State, Country, Accronym, Alias, Link
########################################################################
import pandas as pd
filename = '/content/grid_full.csv'
grid_insts = pd.read_csv(filename)
grid_insts


,ID,Name,City,State,Country,acronym,alias,lat,lng,link,type
0,grid.1001.0,Australian National University,Canberra,Australian Capital Territory,Australia,ANU,NaN,-35.277800,149.120500,http://www.anu.edu.au/,Education
1,grid.1002.3,Monash University,Melbourne,Victoria,Australia,NaN,NaN,-37.908300,145.138000,http://www.monash.edu/,Education
2,grid.1003.2,University of Queensland,Brisbane,Queensland,Australia,UQ,NaN,-27.495964,153.009627,http://www.uq.edu.au/,Education
3,grid.1004.5,Macquarie University,Sydney,New South Wales,Australia,NaN,NaN,-33.775259,151.112915,http://mq.edu.au/,Education
4,grid.1005.4,UNSW Sydney,Sydney,New South Wales,Australia,UNSW,University of New South Wales,-33.917731,151.230964,https://www.unsw.edu.au/,Education
...,...,...,...,...,...,...,...,...,...,...,...
98327,grid.508497.7,Severn Glocon Group (United Kingdom),Gloucester,NaN,United Kingdom,NaN,NaN,51.830360,-2.273422,https://www.severnglocon.com/,Company
98328,grid.508498.8,Universalbeton Heringen (Germany),Heringen,NaN,Germany,NaN,NaN,51.458250,10.861440,http://www.universalbeton.de/,Company
98329,grid.508499.9,University Hospitals of Derby and Burton NHS F...,Uttoxeter,NaN,United Kingdom,UHDB,NaN,52.910665,-1.514036,https://www.uhdb.nhs.uk/,Healthcare
98330,grid.508500.8,Universal-Kugellager-Fabrik (Germany),Berlin,NaN,Germany,UKF,NaN,52.573480,13.329301,http://www.ukf.de/index.php/en/,Company


## Get unclean institution data from Solr
*   NOTE: In the future, ideally use allofplos



In [ ]:
##############################
 ## Get all the institution names from solr (and their frequency):
##############################
import requests
import json
import pandas as pd

####### Get the data (per the link Ben gave me):
a = requests.get("https://api.plos.org/terms?terms.fl=affiliate_facet&terms.limit=1000000&wt=json&indent=true", stream=True).json()

####### Make a df called 'solr_insts' from the data: institution & country columns:
insts = a["terms"]["affiliate_facet"]
institutions = []
countries = []
count = []
for i in range(len(insts)):
    if i % 2 == 0:
        temp = insts[i].split(', ')
        institutions.append(', '.join(temp[0:len(temp)-1]))  ## Some institutions have commas in them
        countries.append(temp[len(temp)-1])
    else:
        count.append(insts[i])

solr_insts = pd.DataFrame({'inst': institutions, 'country': countries, 'count': count})

####### Print out:
solr_insts


In [ ]:
sum(list(solr_insts['count']))


1189751

## Get unclean institution data from EM and Marketing Cloud (via Sisense):


In [1]:
########################################################################
 ## Bring in pre-created raw/unclean data from EM & Marketing Cloud (opt-ins)
   # I queried EM and Sisense directly to get the authors, editors, reviewers and opt-ins data (BQ only has authors for now). 
      # Rows are already de-duplicated.
########################################################################
import pandas as pd
filename = '/content/inst_country_city_optins_added.csv'
em_insts = pd.read_csv(filename)
em_insts = em_insts[em_insts['Institute'].notnull()].reset_index(drop = True)
em_insts

########################################################################
######## DISABLED: GETTING DATA FROM BIGQUERY IS NOT NEEDED FOR NOW: 
   # NOTE: In the future we should get this data directly from BigQuery. Right now, Editor & Reviewer contact info is not in BigQuery, only EM.
     # Per Yanwu: The editor and reviews are not in BQ yet. In EM, the editors and reviewers are joined with address table using peopleid, since address tables are not in BQ, you can not get /city/coutry/email data for editors and reviewers.
     # This information will be available at a later date. 
########################################################################
##################################################
 ## Authenticate yourself to get access to BQ:
##################################################
#from google.colab import auth
#auth.authenticate_user()
#print('Authenticated')

###############
 ### Install and import pandas-gbq, define the project id:
###############
#!pip install pandas-gbq
#import pandas_gbq

#project_id = "plos-data"

##################################################
 ## Get author data from BQ:
##################################################
###############
 ### Get author data from BQ
   # NOTE: INSTITUTEID is only relevant for exact matches
   # NOTE: Affiliation and Institute are sometimes different, use both to help add more name variations.
###############
#sql_authors = """
#  SELECT EMAIL, AFFILIATION, INSTITUTE, CITY, COUNTRY
#  FROM PLOSDL.EM_AUTHORS
#"""
#df_authors = pandas_gbq.read_gbq(sql_authors, project_id=project_id)


# **Parse down Solar & EM data. Create main look-up table AND connect Solr & EM institution names to GRID names through comma separated method**


In [ ]:
##################################################
 ## Remove Solr institutions that exactly match GRID names or aliases.
##################################################
temp_grid = grid_insts[['Name', 'alias']].drop_duplicates(subset=['Name'], keep = False)
solr_insts = pd.merge(solr_insts, temp_grid, how='left', left_on= 'inst', right_on = 'Name')
temp_grid = grid_insts[['Name', 'alias']].drop_duplicates(subset=['alias'], keep = False).dropna()
solr_insts = pd.merge(solr_insts, temp_grid, how='left', left_on= 'inst', right_on = 'alias')
solr_insts = solr_insts[(solr_insts['Name_x'].isnull()) & (solr_insts['alias_y'].isnull())].reset_index(drop = True).drop(['Name_x', 'alias_x', 'Name_y', 'alias_y'], axis=1)



##################################################
 ## (a) Use comma separated method for solr_insts (b) Create main look-up table, starting with these rows linked after comma separation
    # NOTE: Aliases can be too generic for using with split method.
##################################################
###############
 ### Create 'split_inst' solr variables. Create main look-up table by matching split_inst on GRID Names:
###############
solr_insts['split_inst'] = solr_insts['inst'].str.split(',').str[0]
temp_grid = grid_insts[['Name', 'ID']].drop_duplicates(subset=['Name'], keep = False)
main_insts = pd.merge(solr_insts, temp_grid, how='left', left_on= 'split_inst', right_on = 'Name')
main_insts = main_insts[main_insts['ID'].notnull()].drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)


###############
 ### Remove matched rows from solr_insts. Then remove unneeded columns from main_insts
###############
temp_connect = main_insts[['inst', 'ID']]
solr_insts = pd.merge(solr_insts, temp_connect, how='left', on = 'inst')
solr_insts = solr_insts[solr_insts['ID'].isnull()].reset_index(drop = True).drop('ID', axis=1)

main_insts = main_insts[['inst', 'ID']]
main_insts.columns = ['inst', 'grid_id']

main_insts


,inst,grid_id
0,"University of Chinese Academy of Sciences, Bei...",grid.410726.6
1,"University of Basel, Basel",grid.6612.3
2,"Harvard Medical School, Boston, Massachusetts",grid.471403.5
3,"Swiss Tropical and Public Health Institute, Basel",grid.416786.a
4,"Fogarty International Center, National Institu...",grid.453035.4
...,...,...
37691,"Örebro University, School of Health and Medica...",grid.15895.30
37692,"Örebro University, Örebro",grid.15895.30
37693,"Østfold Hospital Trust, Fredrikstad",grid.412938.5
37694,"Østfold University College, Halden",grid.446040.2


In [ ]:
##################################################
 ## Remove EM Institutes that exactly match GRID names or aliases.
##################################################
temp_grid = grid_insts[['Name', 'alias']].drop_duplicates(subset=['Name'], keep = False)
em_insts = pd.merge(em_insts, temp_grid, how='left', left_on= 'Institute', right_on = 'Name')
temp_grid = grid_insts[['Name', 'alias']].drop_duplicates(subset=['alias'], keep = False).dropna()
em_insts = pd.merge(em_insts, temp_grid, how='left', left_on= 'Institute', right_on = 'alias')
em_insts = em_insts[(em_insts['Name_x'].isnull()) & (em_insts['alias_y'].isnull())].reset_index(drop = True).drop(['Name_x', 'alias_x', 'Name_y', 'alias_y'], axis=1)



##################################################
 ## (a) Use comma separated method for em_insts (b) Add matches through comma separated method to main_insts
    # NOTE: Aliases and Affiliation can be too generic for using with split method.
##################################################
###############
 ### Create 'split_inst' solr variables. Create main look-up table by matching split_inst on GRID Names:
###############
em_insts['split_inst'] = em_insts['Institute'].str.split(',').str[0]
temp_grid = grid_insts[['Name', 'ID']].drop_duplicates(subset=['Name'], keep = False)
em_insts = pd.merge(em_insts, temp_grid, how='left', left_on= 'split_inst', right_on = 'Name')

temp_main = em_insts[['Institute', 'ID']]
temp_main = temp_main[temp_main['ID'].notnull()]
temp_main.columns = ['inst', 'grid_id']
main_insts = pd.concat([main_insts,temp_main], ignore_index=True)
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first')

em_insts = em_insts[em_insts['ID'].isnull()].drop(['Name', 'ID'], axis = 1).reset_index(drop = True)

em_insts

# **'Clean' and connect Solr & EM institution names to 'clean' GRID names**

In [ ]:
##################################################
 ## Create method to create new column of 'clean' institution names. Apply to EM & solr inst & split insts, along with GRID data. Also use cleaned EM's affiliation
##################################################
###############
 ### Function to clean strings of a dataset's column:
###############
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer

def clean_column(dataset):
    clean = []
    ## Remove stop words from ONLY english or MANY languages: Currently only using english words
#    stop_words = list(set(list(set(stopwords.words('english'))) + list(set(stopwords.words('spanish'))) + list(set(stopwords.words('arabic'))) + list(set(stopwords.words('german'))) + list(set(stopwords.words('french'))) + list(set(stopwords.words('russian')))))
    stop_words = set(stopwords.words("english"))
    for i in range(len(dataset)):
        temp = []
        temp = str(dataset[i]).lower()
        ## Replace punctuation as spaces:
        translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) # map punctuation to space
        temp = temp.translate(translator)
        temp = temp.strip()
        temp = word_tokenize(temp)
        temp = [i for i in temp if not i in stop_words]
        ## get rid of 1 character words:
        temp = [i for i in temp if (len(i) > 1)]
        ######## NOTE: This is where you'd put in code to replace EM words not in the other data to similar word in the other data (import in the csv with word connectors first):
        clean.append(TreebankWordDetokenizer().detokenize(temp))
    return clean


###############
 ### Use the clean_column function on each dataset:
###############
solr_insts['connect_inst'] = clean_column(solr_insts['inst'])
solr_insts['connect_split_inst'] = clean_column(solr_insts['split_inst'])
em_insts['connect_inst'] = clean_column(em_insts['Institute'])
em_insts['connect_split_inst'] = clean_column(em_insts['split_inst'])
em_insts['connect_affiliation'] = clean_column(em_insts['Affiliation'])
grid_insts['connect_inst'] = clean_column(grid_insts['Name'])

solr_insts


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,inst,country,count,split_inst,connect_inst,connect_split_inst
0,University of South Alabama Mitchell Cancer In...,UNITED STATES,749,University of South Alabama Mitchell Cancer In...,university south alabama mitchell cancer insti...,university south alabama mitchell cancer insti...
1,TNO,NETHERLANDS,422,TNO,tno,tno
2,London School of Hygiene and Tropical Medicine...,United Kingdom,388,London School of Hygiene and Tropical Medicine,london school hygiene tropical medicine london,london school hygiene tropical medicine
3,University of Illinois at Urbana-Champaign,UNITED STATES,358,University of Illinois at Urbana-Champaign,university illinois urbana champaign,university illinois urbana champaign
4,University of Michigan,UNITED STATES,348,University of Michigan,university michigan,university michigan
...,...,...,...,...,...,...
619079,"”Dante Pazzanese” Institute of Cardiology, Sao...",Brazil,1,”Dante Pazzanese” Institute of Cardiology,dante pazzanese institute cardiology sao paulo...,dante pazzanese institute cardiology
619080,”Dr. Carol Davila” Teaching Hospital of Nephro...,Romania,1,”Dr. Carol Davila” Teaching Hospital of Nephro...,dr carol davila teaching hospital nephrology b...,dr carol davila teaching hospital nephrology
619081,"”Golgi Cenci” Foundation, Abbiategrasso",Italy,1,”Golgi Cenci” Foundation,golgi cenci foundation abbiategrasso,golgi cenci foundation
619082,"”Stefan cel Mare” University, Faculty of Food ...",Romania,1,”Stefan cel Mare” University,stefan cel mare university faculty food engine...,stefan cel mare university


In [ ]:
##################################################
 ## (a) Use 'cleaned inst' method to connect to GRID names on: solr's inst and split_inst and em's Institute, split_inst and Affiliation:
   # remove all matches from solr or em (respecively) except matched EM 'clean_affiliations'. 
##################################################
###############
 ### Method to connect clean inst to clean GRID, then remove from dataset and add to main_insts data:
###############
def match_remove_insts(dataset, clean_var, raw_var):
    temp_grid = grid_insts[['connect_inst', 'ID']].drop_duplicates(subset=['connect_inst'], keep = False)
    temp_grid.columns = ['GRID_connector', "ID"]
    temp_dataset = pd.merge(dataset, temp_grid, how='left', left_on= clean_var, right_on = 'GRID_connector')
    temp_main = temp_dataset[[raw_var, 'ID']]
    temp_main = temp_main[temp_main['ID'].notnull()]
    temp_main.columns = ['inst', 'grid_id']
    temp_main_data = pd.concat([main_insts,temp_main], ignore_index=True)
    temp_main_data = temp_main_data.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
    temp_dataset = temp_dataset[temp_dataset['ID'].isnull()].drop(['ID', 'GRID_connector'], axis = 1).reset_index(drop = True)
    return temp_dataset, temp_main_data


###############
 ### Use the method: solr's inst and split_inst and em's Institute, split_inst:
###############
#solr_insts, main_insts = match_remove_insts(solr_insts, 'connect_inst', 'inst')
#solr_insts, main_insts = match_remove_insts(solr_insts, 'connect_split_inst', 'inst')
em_insts, main_insts = match_remove_insts(em_insts, 'connect_inst', 'Institute')
em_insts, main_insts = match_remove_insts(em_insts, 'connect_split_inst', 'Institute')


###############
 ### Connect on EM's 'connect_affiliation' but DON'T remove rows from EM:
###############
temp_grid = grid_insts[['connect_inst', 'ID']].drop_duplicates(subset=['connect_inst'], keep = False)
em_insts = pd.merge(em_insts, temp_grid, how='left', left_on= 'connect_affiliation', right_on = 'connect_inst')
temp_main = em_insts[['Institute', 'ID']]
temp_main = temp_main[temp_main['ID'].notnull()]
temp_main.columns = ['inst', 'grid_id']
main_insts = pd.concat([main_insts,temp_main], ignore_index=True)
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
em_insts = em_insts.drop(['ID'], axis = 1)



####################### NOTE: Messed up on the match_remove_insts() for em_insts (creates bad column names). The below is a temporary fix to deal with this error:
em_insts = em_insts.drop(['connect_inst_y', 'ID'], axis = 1)
em_insts.rename(columns={'connect_inst_x':'connect_inst'}, inplace=True)


# Email domain to GRID link matching: over 75 instances of an institute by email domain in EM data

In [ ]:
##################################################
 ## Use domain matching on domains with over 75 instances of 'connect_inst' for em_insts
    # remove company websites by removing GRID['type'] == 'company' (to remove gmail, yahoo, etc)
##################################################
###############
 ### Make the domain connectors for EM (1st email only) and GRID data. Then connect grid's ID onto em_insts
    # remove company websites by removing GRID['type'] == 'company'
###############
### Create 'domain_connect'
em_insts['domain_connect'] = em_insts['Email'].str.split(";",1).str[0]
temp_grid = grid_insts[grid_insts['type'] != "Company"]
temp_grid['domain_connect'] = temp_grid['link'].str.split("www.",1).str[1].str[:-1]
temp_grid = temp_grid[['domain_connect', 'ID']].dropna().drop_duplicates(subset=['domain_connect'], keep = False)
em_insts = pd.merge(em_insts, temp_grid, how='left', on=['domain_connect'])

###############
 ### Create a temp dataset, de-duplicating by email domain.
###############
temp_domain_data = em_insts.drop_duplicates(subset=['Email'], keep = 'first')
temp_domain_data['concat'] = temp_domain_data['domain_connect'] + temp_domain_data['connect_inst']

###############
 ### Find and limit temp dataset to institute by domain connections with over 75 unique email instances:
###############
import nltk
dom_agg = nltk.FreqDist(temp_domain_data['concat'])
dom_agg = dict((k, v) for k, v in dom_agg.items() if v > 74)              ######################### JUST CHANGE THIS '75' TO MAKE REDO FOR AT LEAST 50 INSTANCES
dom_list = list(dom_agg.keys())
temp_domain_data = temp_domain_data[temp_domain_data['concat'].isin(dom_list)]

###############
 ### Put inst - domain connections with over 75 unique email instances onto main_insts
###############
to_main_data = temp_domain_data[['Institute', 'ID']]
to_main_data = to_main_data[to_main_data['ID'].notnull()]
to_main_data.columns = ['inst', 'grid_id']
main_insts = pd.concat([main_insts,to_main_data], ignore_index=True)
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)

###############
 ### Remove inst - domain connections from EM:
###############
em_insts = em_insts[~em_insts['Institute'].isin(to_main_data['inst'])].reset_index(drop = True).drop('ID', axis=1)

main_insts


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,inst,grid_id
0,"University of Chinese Academy of Sciences, Bei...",grid.410726.6
1,"University of Basel, Basel",grid.6612.3
2,"Harvard Medical School, Boston, Massachusetts",grid.471403.5
3,"Swiss Tropical and Public Health Institute, Basel",grid.416786.a
4,"Fogarty International Center, National Institu...",grid.453035.4
...,...,...
113579,Norwegian University of Life SciencesNorwegian...,grid.19477.3c
113580,University of Bergen & Haukeland University Ho...,grid.7914.b
113581,"Faculty of Landscape Planning, Horticulture an...",grid.6341.0
113582,PFH Private Hochschule Gottingen,grid.462770.0


# Option to save data and export back in before using Dimensions

In [ ]:
##################################################
 ## Export/download the data
   ############ RECOMMENDED: Google colab can disconnect and reset if left idle for too long (e.g. doing Dimensions stuff overnight)
##################################################
###############
 ### Download main_insts
###############
from google.colab import files
main_insts.to_csv('main_insts.csv')
files.download('main_insts.csv')

###############
 ### Download em_insts
###############
em_insts.to_csv('em_insts.csv')
files.download('em_insts.csv')

###############
 ### Download solr_insts
###############
solr_insts.to_csv('solr_insts.csv')
files.download('solr_insts.csv')

###############
 ### Download grid_insts (in case you want the inst_connect column already there)
###############
#grid_insts.to_csv('grid_insts.csv')
#files.download('grid_insts.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##################################################
 ## Import back in the data
   # Remove the first column because it loads out and in the index
##################################################
import pandas as pd
f = '/content/grid_insts.csv'
grid_insts = pd.read_csv(f)
grid_insts = grid_insts.iloc[:,1:]
f = '/content/main_insts.csv'
main_insts = pd.read_csv(f)
main_insts = main_insts.iloc[:,1:]
f = '/content/em_insts.csv'
em_insts = pd.read_csv(f)
em_insts = em_insts.iloc[:,1:]
f = '/content/solr_insts.csv'
solr_insts = pd.read_csv(f)
solr_insts = solr_insts.iloc[:,1:]

em_insts


# Use Dimensions extract_affiliation in batches to get em and solr matches

In [ ]:
##################################################
 ## Before using Dimensions: Remove instances on em and solr that already exist on main_insts
     # If the inst names are already there, no need to look up with country and/or city
 ## NOTE: You can use this after Dimensions & fuzzy matching too: It cleans the em and solr data from existing main_insts insts.
##################################################
solr_insts = pd.merge(solr_insts, main_insts, how='left', on = 'inst')
solr_insts = solr_insts[solr_insts['grid_id'].isnull()].drop('grid_id', axis = 1).reset_index(drop=True)
em_insts = pd.merge(em_insts, main_insts, how='left', left_on= 'Institute', right_on = 'inst')
em_insts = em_insts[em_insts['grid_id'].isnull()].drop(['inst', 'grid_id'], axis = 1).reset_index(drop=True)
solr_insts


,inst,country,count,split_inst,connect_inst,connect_split_inst,inst_no_punct,country_no_punct
0,"Freelance Science Writer, Sherborn, Massachusetts",United States of America,91,Freelance Science Writer,freelance science writer sherborn massachusetts,freelance science writer,Freelance Science Writer Sherborn Massachusetts,United States of America
1,"Epicentre, Paris",France,81,Epicentre,epicentre paris,epicentre,Epicentre Paris,France
2,Fondation Raoul Follereau,FRANCE,69,Fondation Raoul Follereau,fondation raoul follereau,fondation raoul follereau,Fondation Raoul Follereau,FRANCE
3,Institute for Health & the Environment,UNITED STATES,68,Institute for Health & the Environment,institute health environment,institute health environment,Institute for Health the Environment,UNITED STATES
4,Raymond M. Alf Museum of Paleontology,United States of America,67,Raymond M. Alf Museum of Paleontology,raymond alf museum paleontology,raymond alf museum paleontology,Raymond M Alf Museum of Paleontology,United States of America
...,...,...,...,...,...,...,...,...
140353,“Victor Babes” University of Medicine and Phar...,Romania,1,“Victor Babes” University of Medicine and Phar...,victor babes university medicine pharmacy dept...,victor babes university medicine pharmacy,Victor Babes University of Medicine and Phar...,Romania
140354,“Victor Babes” University of Medicine and Phar...,Romania,1,“Victor Babes” University of Medicine and Phar...,victor babes university medicine pharmacy dept...,victor babes university medicine pharmacy,Victor Babes University of Medicine and Phar...,Romania
140355,"“Vittorio Emanuele” Hospital, Liver Unit, Catania",Italy,1,“Vittorio Emanuele” Hospital,vittorio emanuele hospital liver unit catania,vittorio emanuele hospital,Vittorio Emanuele Hospital Liver Unit Catania,Italy
140356,"”Dante Pazzanese” Institute of Cardiology, Sao...",Brazil,1,”Dante Pazzanese” Institute of Cardiology,dante pazzanese institute cardiology sao paulo...,dante pazzanese institute cardiology,Dante Pazzanese Institute of Cardiology Sao...,Brazil


In [ ]:
##################################################
 ## Use Dimensions’ extract_affiliation method on (a) raw Solar name (b) Solar name after comma separated method
##################################################
###############
 ### Jarrett log on to Dimensions: REDACTED
###############
username = # *** REDACTED ***
password = # *** REDACTED ***
endpoint = "https://app.dimensions.ai"

###############
 ### Need to install at beginning of each session:
###############
!pip install dimcli

###############
 ### Easy sample query with iterations to get full results:
###############
import dimcli
dimcli.login(username, password, endpoint)
dsl = dimcli.Dsl()




##################################################
 ## Method to use Dimensions' extract_affiliation() with a 199 batch with solr & EM:
   # Use solr's inst & connect_inst; use EM's Institute & connect_inst (split inst is too risky).
##################################################
import json
import time
import numpy as np
import pandas as pd

def dimensions_batch(unclean_data, merged_inst_name, raw_inst_name):
    ### NOTE: I use batches of length 199 for dimensions: https://docs.dimensions.ai/dsl/functions.html#function-extract-affiliations
    count = 0
    final_data = pd.DataFrame(columns = ['inst', 'grid_id'])
    for i in range(int(len(unclean_data)/199)-1):
        temp_batch = ','.join(unclean_data[merged_inst_name][count:(count+199)])
        data = dsl.query(f"""extract_affiliations(json=[{temp_batch}])""")
        temp_data = []
        for j in range(199):
        ### If there is a result for institution put in grid_id, if not put in 'NaN' (need for symmetry with inital dataset):
            if len(data.json['results'][j]['matches'][0]['institutes']) > 0:
                temp_data.append(json.dumps(data['results'][j]['matches'][0]['institutes'][0]['institute']['id'])[1:-1])
            else:
                temp_data.append(np.nan)
        temp_dataframe = pd.DataFrame({'inst': list(unclean_data[raw_inst_name][count:(count+199)]), 'grid_id': temp_data})
        final_data = pd.concat([final_data, temp_dataframe], ignore_index=True)
        time.sleep(2.05)
        count += 199
    return final_data



##################################################
 ## Use the dimensions_batch() function on each data/var combination:
   # Use on solr's inst & connect_inst; use EM's Institute, connect_inst and Affiliation
   # Take out punctuation and merge variables into pre-combined json format.
##################################################
###############
 ### Use dimensions_batch() on Solr's inst
###############
### Create 'merged' field with no punctuations and use the dimensions_batch function
#solr_insts = solr_insts[solr_insts['inst'].notnull()].reset_index(drop=True)  # remove nas from inst (produces errors in .join)
#solr_insts['inst_no_punct'] = solr_insts['inst'].str.replace('[^\w\s]',' ')
#solr_insts['country_no_punct'] = solr_insts['country'].str.replace('[^\w\s]',' ')
#solr_insts['merged'] = '{"affiliation":' + ' "' + solr_insts['inst_no_punct'] + " " + solr_insts['country_no_punct'] + '"}'
##solr_insts_2 = solr_insts.iloc[0:400,:].reset_index(drop=True)   ## For a smaller sample, add '_2' to end of next row's 1st entry
#cleaned_solr_inst = dimensions_batch(solr_insts, "merged", "inst")
### Connect matches to main_insts:
#new_connects = cleaned_solr_inst[cleaned_solr_inst['grid_id'].notnull()]
#main_insts = pd.concat([main_insts, new_connects], ignore_index=True)
#main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
### Remove matches from solr_insts:
#new_connects = new_connects.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
#solr_insts = pd.merge(solr_insts, new_connects, how='left', on = 'inst')
#solr_insts = solr_insts[solr_insts['grid_id'].isnull()].drop('grid_id', axis = 1).reset_index(drop=True)

###############
 ### Use dimensions_batch() on Solr's connect_inst (cleaned insts):
###############
### Create 'merged' field with clean data and use the dimensions_batch function
#solr_insts['country_no_punct'] = solr_insts['country'].str.replace('[^\w\s]',' ')
#solr_insts['merged'] = '{"affiliation":' + ' "' + solr_insts['connect_inst'] + " " + solr_insts['country_no_punct'] + '"}'
##solr_insts_2 = solr_insts.iloc[0:400,:].reset_index(drop=True)   ## For a smaller sample, add '_2' to end of next row's 1st entry
#cleaned_solr_connect_inst = dimensions_batch(solr_insts, "merged", "inst")
### Connect matches to main_insts:
#new_connects = cleaned_solr_connect_inst[cleaned_solr_connect_inst['grid_id'].notnull()]
#main_insts = pd.concat([main_insts, new_connects], ignore_index=True)
#main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
### Remove matches from solr_insts:
#new_connects = new_connects.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
#solr_insts = pd.merge(solr_insts, new_connects, how='left', on = 'inst')
#solr_insts = solr_insts[solr_insts['grid_id'].isnull()].drop('grid_id', axis = 1).reset_index(drop=True)


###############
 ### Use dimensions_batch() on Use dimensions_batch() on EM's Institute:
   # Need to change Country and City for Em data since there's many N/A's
###############
### Create 'merged' field where Country and City NAs = "", with no punctuations in inst city or country, and use the dimensions_batch function
#em_insts = em_insts[em_insts['Institute'].notnull()].reset_index(drop=True)
#temp_em_insts = em_insts.fillna("")
#temp_em_insts.loc[temp_em_insts.Country == "N\A", 'Country'] = ""
#temp_em_insts.loc[temp_em_insts.City == "N\A", 'City'] = ""
#emp_em_insts['inst_no_punct'] = temp_em_insts['Institute'].str.replace('[^\w\s]',' ')
#temp_em_insts['Country'] = temp_em_insts['Country'].str.replace('[^\w\s]',' ')
#temp_em_insts['City'] = temp_em_insts['City'].str.replace('[^\w\s]',' ')
#temp_em_insts['no_duplicates'] = temp_em_insts['inst_no_punct'] + temp_em_insts['Country'] + temp_em_insts['City']
#temp_em_insts = temp_em_insts.drop_duplicates(subset=['no_duplicates'], keep = 'first').reset_index(drop = True)
### Create 'merged' field with clean data and use the dimensions_batch function
#temp_em_insts['merged'] = '{"affiliation":' + ' "' + temp_em_insts['inst_no_punct'] + " " + temp_em_insts['Country'] + " " + temp_em_insts['City'] + '"}'
##temp_em_insts_2 = temp_em_insts.iloc[0:600,:].reset_index(drop=True)   ## For a smaller sample, add '_2' to end of next row's 1st entry
#cleaned_em_inst = dimensions_batch(temp_em_insts, "merged", "Institute")
### Connect matches to main_insts:
#new_connects = cleaned_em_inst[cleaned_em_inst['grid_id'].notnull()]
#main_insts = pd.concat([main_insts, new_connects], ignore_index=True)
#main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
### Remove matches from em_insts:
#new_connects = new_connects.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
#em_insts = pd.merge(em_insts, new_connects, how='left', left_on= 'Institute', right_on = 'inst')
#em_insts = em_insts[em_insts['grid_id'].isnull()].drop(['inst', 'grid_id'], axis = 1).reset_index(drop=True)

###############
 ### Use dimensions_batch() on EM's connect_inst (cleaned insts):
###############
### Create 'merged' field where Country and City NAs = "", with no punctuations in city or country, and use the dimensions_batch function
#em_insts = em_insts[em_insts['Institute'].notnull()].reset_index(drop=True)
#temp_em_insts = em_insts.fillna("")
#temp_em_insts.loc[temp_em_insts.Country == "N\A", 'Country'] = ""
#temp_em_insts.loc[temp_em_insts.City == "N\A", 'City'] = ""
#temp_em_insts['Country'] = temp_em_insts['Country'].str.replace('[^\w\s]',' ')
#temp_em_insts['City'] = temp_em_insts['City'].str.replace('[^\w\s]',' ')
#temp_em_insts['no_duplicates'] = temp_em_insts['connect_inst'] + temp_em_insts['Country'] + temp_em_insts['City']
#temp_em_insts = temp_em_insts.drop_duplicates(subset=['no_duplicates'], keep = 'first').reset_index(drop = True)
### Create 'merged' field with clean data and use the dimensions_batch function
#temp_em_insts['merged'] = '{"affiliation":' + ' "' + temp_em_insts['connect_inst'] + " " + temp_em_insts['Country'] + " " + temp_em_insts['City'] + '"}'
#temp_em_insts = temp_em_insts.iloc[0:600,:].reset_index(drop=True)   ## For a smaller sample, add '_2' to end of next row's 1st entry
#cleaned_em_connect_inst = dimensions_batch(temp_em_insts, "merged", "Institute")
### Connect matches to main_insts:
#new_connects = cleaned_em_connect_inst[cleaned_em_connect_inst['grid_id'].notnull()]
#main_insts = pd.concat([main_insts, new_connects], ignore_index=True)
#main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
### Remove matches from em_insts:
#new_connects = new_connects.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
#em_insts = pd.merge(em_insts, new_connects, how='left', left_on= 'Institute', right_on = 'inst')
#em_insts = em_insts[em_insts['grid_id'].isnull()].drop(['inst', 'grid_id'], axis = 1).reset_index(drop=True)

###############
 ### Use dimensions_batch() on Use dimensions_batch() on EM's clean_affiliation (cleaned affiliations):
###############
### Create 'merged' field where Country and City NAs = "", with no punctuations in city or country, and use the dimensions_batch function
temp_em_insts = em_insts[em_insts['connect_affiliation'].notnull()].reset_index(drop=True)
temp_em_insts = em_insts.fillna("")
temp_em_insts.loc[temp_em_insts.Country == "N\A", 'Country'] = ""
temp_em_insts.loc[temp_em_insts.City == "N\A", 'City'] = ""
temp_em_insts['Country'] = temp_em_insts['Country'].str.replace('[^\w\s]',' ')
temp_em_insts['City'] = temp_em_insts['City'].str.replace('[^\w\s]',' ')
temp_em_insts['no_duplicates'] = temp_em_insts['connect_affiliation'] + temp_em_insts['Country'] + temp_em_insts['City']
temp_em_insts = temp_em_insts.drop_duplicates(subset=['no_duplicates'], keep = 'first').reset_index(drop = True)
### Create 'merged' field with clean data and use the dimensions_batch function
temp_em_insts['merged'] = '{"affiliation":' + ' "' + temp_em_insts['connect_affiliation'] + " " + temp_em_insts['Country'] + " " + temp_em_insts['City'] + '"}'
#temp_em_insts = temp_em_insts.iloc[0:600,:].reset_index(drop=True)   ## For a smaller sample, add '_2' to end of next row's 1st entry
cleaned_em_connect_aff = dimensions_batch(temp_em_insts, "merged", "Affiliation")
### Connect matches to main_insts:
new_connects = cleaned_em_connect_aff[cleaned_em_connect_aff['grid_id'].notnull()]
main_insts = pd.concat([main_insts, new_connects], ignore_index=True)
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)



# Email domain to GRID link matching: over 50 instances of an institute by email domain in EM data

In [ ]:
##################################################
 ## Use domain matching on domains with over 50 instances of 'connect_inst' for em_insts
##################################################
###############
 ### Make the domain connectors for EM (1st email only) and GRID data. Then connect grid's ID onto em_insts
###############
### Create 'domain_connect'
em_insts['domain_connect'] = em_insts['Email'].str.split(";",1).str[0]
em_insts['domain_connect'] = em_insts['domain_connect'].str.split("@",1).str[1]
temp_grid = grid_insts[grid_insts['type'] == "Education"]
temp_grid['domain_connect'] = temp_grid['link'].str.split("www.",1).str[1].str[:-1]
temp_grid = temp_grid[['domain_connect', 'ID']].dropna().drop_duplicates(subset=['domain_connect'], keep = False)
em_insts = pd.merge(em_insts, temp_grid, how='left', on=['domain_connect'])

###############
 ### Create a temp dataset, de-duplicating by email domain.
###############
temp_domain_data = em_insts.drop_duplicates(subset=['Email'], keep = 'first')
temp_domain_data['concat'] = temp_domain_data['domain_connect'] + temp_domain_data['connect_inst']

###############
 ### Find and limit temp dataset to institute by domain connections with over 100 unique email instances:
###############
import nltk
dom_agg = nltk.FreqDist(temp_domain_data['concat'])
dom_agg = dict((k, v) for k, v in dom_agg.items() if v > 49)              ######################### JUST CHANGE THIS '49' TO GO LOWER.
dom_list = list(dom_agg.keys())
temp_domain_data = temp_domain_data[temp_domain_data['concat'].isin(dom_list)]

###############
 ### Put inst - domain connections onto main_insts
###############
to_main_data = temp_domain_data[['Institute', 'ID']]
to_main_data = to_main_data[to_main_data['ID'].notnull()]
to_main_data.columns = ['inst', 'grid_id']
main_insts = pd.concat([main_insts,to_main_data], ignore_index=True)
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)

###############
 ### Remove inst - domain connections from EM:
###############
em_insts = em_insts[~em_insts['Institute'].isin(to_main_data['inst'])].reset_index(drop = True).drop('ID', axis=1)


em_insts

# Use fuzzy matching (with KNN) to connect EM & Solr to GRID

In [ ]:
##################################################
  ## Fast fuzzy matching
    # NOTE: Found a way quicker way than fuzzywuzzy to do fuzzy matching here: https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536
        # Calculates tf-idfs on words & bi-grams of GRID insts, then uses k nearest neighborhors to calculate similarity.
    # NOTE: Using clean institutions got a little weird so I'm only use the non-clean inst names.
    ### Based on some research/tests, it looks like Solr is good using a match under .76. em insts is *much* murkier and should only use under .41
##################################################
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.neighbors import NearestNeighbors

def kneighbors_fuzzy_match(unclean_list, clean_list, grid_id_connect):
    vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1, 2), lowercase=False)
    tfidf = vectorizer.fit_transform(clean_list)
    nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
    ### Matching query:
    def getNearestN(query):
        queryTFIDF_ = vectorizer.transform(query)
        distances, indices = nbrs.kneighbors(queryTFIDF_)
        return distances, indices
    distances, indices = getNearestN(unclean_list)
    unique_org = list(unclean_list) #need to convert back to a list
    matches = []
    for i,j in enumerate(indices):
        temp = [round(distances[i][0],2), clean_list.values[j][0], unique_org[i], str(grid_id_connect[j]).split(' ')[4][:-6]]
        matches.append(temp)
    matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name (clean)','inst', 'grid_id'])
    matches = matches.sort_values('Match confidence (lower is better)')
    return matches



##################################################
  ## Use the kneighbors_fuzzy_match() method, then add to main_insts and remove from solr or EM
     ## Do the following: solr's inst, clean_inst, then split_inst ... and EM's Institute, clean_inst, then clean_affiliation.
     ## NOTE: NEED TO DEFINE WHAT IS A HIGH CONFIDENCE MATCH BEFORE PUTTING INTO MAIN DATA (currently using under .76)
##################################################
###############
 ### Define grid insts to use (unclean and clean)
###############
temp_grid = grid_insts[['Name', 'ID']].drop_duplicates(subset=['Name'], keep = False).reset_index(drop=True)


###############
 ### fuzzy match unclean solr insts:
###############
#### Use kneighbors_fuzzy_match() to get fuzzy matches
solr_insts_2 = list(set(list(solr_insts['inst'])))
solr_fuzzy_connect = kneighbors_fuzzy_match(solr_insts_2, temp_grid['Name'], temp_grid['ID'])
#### Connect matches to main_insts:
new_connects = solr_fuzzy_connect[(solr_fuzzy_connect['Match confidence (lower is better)'] < 0.76) & (solr_fuzzy_connect['Match confidence (lower is better)'] > 0) & (solr_fuzzy_connect['inst'] != "")]
new_connects = new_connects[['inst', 'grid_id']].drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
main_insts = pd.concat([main_insts, new_connects], ignore_index=True)
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
#### Remove matches from solr_insts:
solr_insts = pd.merge(solr_insts, new_connects, how='left', on = 'inst')
solr_insts = solr_insts[solr_insts['grid_id'].isnull()].drop('grid_id', axis = 1).reset_index(drop=True)


###############
 ### fuzzy match unclean em insts:
###############
#### Use kneighbors_fuzzy_match() to get fuzzy matches
em_insts_2 = em_insts.drop_duplicates(subset=['Institute'], keep = 'first').reset_index(drop = True)
em_insts_2 = em_insts_2['Institute']
em_fuzzy_connect = kneighbors_fuzzy_match(em_insts_2, temp_grid['Name'], temp_grid['ID'])
#### Connect matches to main_insts:
new_connects = em_fuzzy_connect[(em_fuzzy_connect['Match confidence (lower is better)'] < 0.41) & (em_fuzzy_connect['Match confidence (lower is better)'] > 0) & (em_fuzzy_connect['inst'] != "")]
new_connects = new_connects[['inst', 'grid_id']].drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
main_insts = pd.concat([main_insts, new_connects], ignore_index=True)
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
#### Remove matches from em_insts:
em_insts = pd.merge(em_insts, new_connects, how='left', left_on= 'Institute', right_on = 'inst')
em_insts = em_insts[em_insts['grid_id'].isnull()].drop('grid_id', 'inst', axis = 1).reset_index(drop=True)


em_insts


# By hand connecting unconnected rows

In [ ]:
##################################################
  ## By hand connections for top unconnected solr_insts
##################################################
###############
 ### Found the GRIDs (for those that could actually be connected) for the top 25. Attach to final lookup list:
###############
#solr_insts.iloc[0:25]
#solr_add_ons = pd.DataFrame([['DOE Pacific Northwest National Laboratory', 'grid.451303.0'], ["Charité, Campus Benjamin Franklin", 'grid.6363.0'], ['Department of Molecular Genetics and Microbiology, Duke University Medical Center, Durham, North Carolina', 'grid.414179.e'], ['Department of Biological Engineering, Massachusetts Institute of Technology, Cambridge, Massachusetts', 'grid.116068.8'], ['Vaccine and Infectious Disease Division, Fred Hutchinson Cancer Research Center, Seattle, Washington', 'grid.270240.3'], ['Department of Pathology, University of Texas Medical Branch, Galveston, Texas', 'grid.176731.5'], ['Management Office for Health Data, China Medical University Hospital, Taichung', 'grid.411508.9'], ['Department of Molecular and Cellular Biology, Baylor College of Medicine, Houston, Texas', 'grid.39382.33'], ['Department of Pediatrics, Baylor College of Medicine, Houston, Texas, Houston, Texas', 'grid.39382.33'], ['Sanjay Gandhi Medical Institute', 'grid.263138.d'], ['Department of Infectious Disease Epidemiology, London School of Hygiene and Tropical Medicine, London', 'grid.8991.9'], ['Faculty of Epidemiology and Population Health, London School of Hygiene and Tropical Medicine, London', 'grid.8991.9'], ['Department of Molecular Virology and Microbiology, Baylor College of Medicine, Houston, Texas', 'grid.39382.33'], ["Institut d'Investigacions Biomediques de Barcelona", 'grid.420258.9'], ["IRCCS E. Medea", 'grid.420417.4'], ["Institute of Epidemiology and Preventive Medicine, College of Public Health, National Taiwan University, Taipei", 'grid.19188.39'], ["Duke Global Health Institute, Duke University, Durham, North Carolina", 'grid.26009.3d'], ['Department of Neuroscience, Baylor College of Medicine, Houston, Texas', 'grid.39382.33'], ["Department of Global Health and Social Medicine, Harvard Medical School, Boston, Massachusetts", 'grid.471403.5']], columns = ['inst', 'grid_id'])
#main_insts = pd.concat([main_insts, solr_add_ons], ignore_index=True)



##################################################
  ## By hand connections for top unconnected em_insts
##################################################
###############
 ### Get the frequency of unconnected em_insts (solr already has counts). Attach to final lookup list:
###############
#import nltk
#em_insts['Inst_by_country'] = em_insts['Institute'] + " : " + em_insts['Country']
#em_count = nltk.FreqDist(em_insts['Inst_by_country'])
#em_count = dict((k, v) for k, v in em_count.items() if v > 50)
#em_count = sorted(em_count.items(), key=lambda x: x[1], reverse = True)
#em_count


###############
 ### Find GRIDs for unconnected over 50
###############
em_add_ons = pd.DataFrame([["FHI 360", "grid.245835.d"], ["INMI Lazzaro Spallanzani IRCCS", "grid.419423.9"], ["INRA Centre Val de Loire", "grid.418065.e"], ["Uniwersytet Warminsko-Mazurski", "grid.412607.6"], ["Shandong Qianfoshan Hospital", "grid.452422.7"], ["INRAE", "grid.507621.7"], ["Guangzhou Military General Hospital", "grid.413435.4"], ["Ahvaz Jondishapour University of Medical Sciences", "grid.411230.5"], ["Wageningen UR", "grid.4818.5"], ["Guangdong Hospital of Traditional Chinese Medicine", "grid.413402.0"], ["Changzhou First People's Hospital", "grid.490563.d"], ["College of Medicine, Korea University", "grid.222754.4"], ["Hellenic Center for Marine Research", "grid.410335.0"], ["INRA Centre de Bordeaux-Aquitaine", "grid.462308.b"], ["Biologicke centrum Akademie Ved Ceske Republiky", "grid.418338.5"], ["Children's Hospital of Shanghai", "grid.415625.1"], ["U.S. Fish and Wildlife Service", "grid.245835.d"], ["MRC/UVRI and LSHTM Uganda Research Unit", "grid.415861.f"], ["Zibo Central Hospital", "grid.477019.c"], ["GHESKIO", "grid.456968.0"], ["Azienda Socio Sanitaria Territoriale Grande Ospedale Metropolitano Niguarda", "grid.416200.1"], ["Rumah Sakit Dr Cipto Mangunkusumo", "grid.487294.4"], ["KEMRI/CDC", "grid.33058.3d"], ["Azienda Socio Sanitaria Territoriale degli Spedali Civili di Brescia", "grid.412725.7"], ["ICRISAT", "grid.419337.b"], ["Galilee Medical Center", "grid.415839.2"], ["IBMB-CSIC", "grid.428973.3"], ["Wageningen Bioveterinary Research", "grid.4818.5"], ["Akademia Wychowania Fizycznego imienia Jerzego Kukuczki w Katowicach", "grid.445174.7"], ["UMKC", "grid.266756.6"], ["Hualien Tzu Chi Hospital", "grid.464578.c"]], columns = ['inst', 'grid_id'])
main_insts = pd.concat([main_insts, em_add_ons], ignore_index=True)
main_insts




,inst,grid_id
0,"University of Chinese Academy of Sciences, Bei...",grid.410726.6
1,"University of Basel, Basel",grid.6612.3
2,"Harvard Medical School, Boston, Massachusetts",grid.471403.5
3,"Swiss Tropical and Public Health Institute, Basel",grid.416786.a
4,"Fogarty International Center, National Institu...",grid.453035.4
...,...,...
660985,IBMB-CSIC,grid.428973.3
660986,Wageningen Bioveterinary Research,grid.4818.5
660987,Akademia Wychowania Fizycznego imienia Jerzego...,grid.445174.7
660988,UMKC,grid.266756.6


# By-hand removal of common insts on main_insts that shouldn't be there 

In [ ]:
##################################################
  ## By-hand removal of common insts on main_insts
##################################################
###############
 ### There was one empty inst, so remove. Also de-dup and remove where inst is only numbers:
###############
main_insts = main_insts[main_insts['inst'].notnull()]
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
main_insts = main_insts[~main_insts['inst'].str.isnumeric()].reset_index(drop = True)

###############
 ### Remove where the character len of inst is '2 or under' or 'over 425'
###############
main_insts['char_len'] = main_insts['inst'].str.len()
main_insts = main_insts[(main_insts['char_len'] > 2) & (main_insts['char_len'] < 425)].drop('char_len', axis=1).reset_index(drop = True)

###############
 ### Common non-'Education' GRID connections (over 90 rows in main_insts) that should be excluded from main_insts list: 
###############
#import nltk
#counts = nltk.FreqDist(main_insts['grid_id'])
#counts = dict((k, v) for k, v in counts.items() if v > 20)
#counts = sorted(counts.items(), key=lambda x: x[1], reverse = True)
#counts = pd.merge(pd.DataFrame(counts), grid_insts, how='left', left_on= 0, right_on = 'ID')
#counts = counts[[0,1,"Name"]]
#counts
##### Look at the name variations for a particular grid:
#main_insts[main_insts['grid_id'] == 'XXXXXXXXXX']

###############
 ### Common non-'Education' GRID connections (over 90 rows in main_insts) that should be excluded from main_insts list: 
###############
grid_ids_to_remove = ["grid.466117.3", "grid.462718.e", "grid.449710.f", "grid.495658.3", "grid.426602.4", "grid.418761.d", "grid.467642.5", "grid.466112.6", "grid.419475.a", "grid.414102.2", "grid.417768.b", "grid.462937.d", "grid.501573.5", "grid.414070.6"]
    # NOTE: These are the grid_ids to remove from main_insts (way too generic)
      # These are their names: ["Department of Biological Sciences", "Department of Virology", "University Hospital", "Institute of Biology", "Institute of Virology", "Institute of Immunology", "Center for Global Health", "Department of Mathematical Sciences", "National Institute on Aging", "Department of Health", "Center for Cancer Research", "Computer Science Department", "Department of Archaeology", "Children's Hospital"]
main_insts = main_insts[~main_insts['grid_id'].isin(grid_ids_to_remove)].reset_index(drop = True)

###############
 ### Based on the look-up, remove rows where the grid_id starts with 'Department of'
###############
department_to_remove = grid_insts[grid_insts['Name'].str.startswith('Department of', na=False)]
department_to_remove = list(department_to_remove["ID"])
main_insts = main_insts[~main_insts['grid_id'].isin(department_to_remove)].reset_index(drop = True)

###############
 ### Download main_insts again:
###############
#from google.colab import files
#main_insts.to_csv('main_insts.csv')
#files.download('main_insts.csv')

main_insts



,inst,grid_id,merging
0,"University of Chinese Academy of Sciences, Bei...",grid.410726.6,"University of Chinese Academy of Sciences, Bei..."
1,"University of Basel, Basel",grid.6612.3,"University of Basel, Basel : grid.6612.3"
2,"Harvard Medical School, Boston, Massachusetts",grid.471403.5,"Harvard Medical School, Boston, Massachusetts ..."
3,"Swiss Tropical and Public Health Institute, Basel",grid.416786.a,"Swiss Tropical and Public Health Institute, Ba..."
4,"Fogarty International Center, National Institu...",grid.453035.4,"Fogarty International Center, National Institu..."
...,...,...,...
655440,IBMB-CSIC,grid.428973.3,IBMB-CSIC : grid.428973.3
655441,Wageningen Bioveterinary Research,grid.4818.5,Wageningen Bioveterinary Research : grid.4818.5
655442,Akademia Wychowania Fizycznego imienia Jerzego...,grid.445174.7,Akademia Wychowania Fizycznego imienia Jerzego...
655443,UMKC,grid.266756.6,UMKC : grid.266756.6


# Use fuzzy matching (with KNN) to connect EM & Solr to main lookup table (main_insts)

In [ ]:
##################################################
  ## Fuzzy matching solr & em on main_insts:
    # NOTE: Do on solr & EM unclean insts.
      # NOTE: Make the threshold stricter than the 1st go. Based on research: under .66 for solr and under .36 for em
##################################################
###############
 ### Fast fuzzy matching (same as above):
###############
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.neighbors import NearestNeighbors
def kneighbors_fuzzy_match(unclean_list, clean_list, grid_id_connect):
    vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1, 2), lowercase=False)
    tfidf = vectorizer.fit_transform(clean_list)
    nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
    ### Matching query:
    def getNearestN(query):
        queryTFIDF_ = vectorizer.transform(query)
        distances, indices = nbrs.kneighbors(queryTFIDF_)
        return distances, indices
    distances, indices = getNearestN(unclean_list)
    unique_org = list(unclean_list) #need to convert back to a list
    matches = []
    for i,j in enumerate(indices):
        temp = [round(distances[i][0],2), clean_list.values[j][0], unique_org[i], str(grid_id_connect[j]).split(' ')[4][:-6]]
        matches.append(temp)
    matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name (clean)','inst', 'grid_id'])
    matches = matches.sort_values('Match confidence (lower is better)')
    return matches

###############
 ### Fuzzy matching solr inst on the insts in main_insts:
###############
#### Use kneighbors_fuzzy_match() to get fuzzy matches
#solr_insts_2 = list(set(list(solr_insts['inst'])))
#solr_fuzzy_connect = kneighbors_fuzzy_match(solr_insts_2, main_insts['inst'], main_insts['grid_id'])
#solr_fuzzy_connect
#### Connect matches to main_insts:
#new_connects = solr_fuzzy_connect[(solr_fuzzy_connect['Match confidence (lower is better)'] < 0.66) & (solr_fuzzy_connect['Match confidence (lower is better)'] > 0) & (solr_fuzzy_connect['inst'] != "")]
#new_connects = new_connects[['inst', 'grid_id']].drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
#main_insts = pd.concat([main_insts, new_connects], ignore_index=True)
#main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
#### Remove matches from solr_insts:
#solr_insts = pd.merge(solr_insts, new_connects, how='left', on = 'inst')
#solr_insts = solr_insts[solr_insts['grid_id'].isnull()].drop('grid_id', axis = 1).reset_index(drop=True)


###############
 ### Fuzzy matching em inst on the insts in main_insts:
###############
#### Use kneighbors_fuzzy_match() to get fuzzy matches
em_insts_2 = em_insts.drop_duplicates(subset=['Institute'], keep = 'first').reset_index(drop = True)
em_insts_2 = em_insts_2['Institute']
em_fuzzy_connect = kneighbors_fuzzy_match(em_insts_2, main_insts['inst'], main_insts['grid_id'])
em_fuzzy_connect
#### Connect matches to main_insts:
new_connects = em_fuzzy_connect[(em_fuzzy_connect['Match confidence (lower is better)'] < 0.36) & (em_fuzzy_connect['Match confidence (lower is better)'] > 0) & (em_fuzzy_connect['inst'] != "")]
new_connects = new_connects[['inst', 'grid_id']].drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
main_insts = pd.concat([main_insts, new_connects], ignore_index=True)
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
#### Remove matches from em_insts:
em_insts = pd.merge(em_insts, new_connects, how='left', left_on= 'Institute', right_on = 'inst')
em_insts = em_insts[em_insts['grid_id'].isnull()].drop(['grid_id', 'inst'], axis = 1).reset_index(drop=True)
em_insts



# Final removal of inaccurate main_insts

In [ ]:
###################################################
 ## Final removal of main_insts
###################################################
###############
 ### For the new connects from the main_inst fuzzy match, exclude startswith('Department of'
###############
a = main_insts[700000:]
a = a[~a['inst'].str.startswith('Department of', na=False)]
b = main_insts[:700000]
main_insts = pd.concat([b,a], ignore_index=True)

###############
 ### De-dup and remove where inst is only numbers:
###############
main_insts = main_insts[main_insts['inst'].notnull()]
main_insts = main_insts.drop_duplicates(subset=['inst'], keep = 'first').reset_index(drop = True)
main_insts = main_insts[~main_insts['inst'].str.isnumeric()].reset_index(drop = True)

###############
 ### Remove where the character len of inst is '2 or under' or 'over 425'
###############
main_insts['char_len'] = main_insts['inst'].str.len()
main_insts = main_insts[(main_insts['char_len'] > 2) & (main_insts['char_len'] < 425)].drop('char_len', axis=1).reset_index(drop = True)

###############
 ### Common non-'Education' GRID connections (over 90 rows in main_insts) that should be excluded from main_insts list: 
###############
grid_ids_to_remove = ["grid.466117.3", "grid.462718.e", "grid.449710.f", "grid.495658.3", "grid.426602.4", "grid.418761.d", "grid.467642.5", "grid.466112.6", "grid.419475.a", "grid.414102.2", "grid.417768.b", "grid.462937.d", "grid.501573.5", "grid.414070.6"]
    # NOTE: These are the grid_ids to remove from main_insts (way too generic)
      # These are their names: ["Department of Biological Sciences", "Department of Virology", "University Hospital", "Institute of Biology", "Institute of Virology", "Institute of Immunology", "Center for Global Health", "Department of Mathematical Sciences", "National Institute on Aging", "Department of Health", "Center for Cancer Research", "Computer Science Department", "Department of Archaeology", "Children's Hospital"]
main_insts = main_insts[~main_insts['grid_id'].isin(grid_ids_to_remove)].reset_index(drop = True)

###############
 ### Download main_insts again:
###############
#from google.colab import files
#main_insts.to_csv('main_insts.csv')
#files.download('main_insts.csv')

main_insts


,inst,grid_id
0,"University of Chinese Academy of Sciences, Bei...",grid.410726.6
1,"University of Basel, Basel",grid.6612.3
2,"Harvard Medical School, Boston, Massachusetts",grid.471403.5
3,"Swiss Tropical and Public Health Institute, Basel",grid.416786.a
4,"Fogarty International Center, National Institu...",grid.453035.4
...,...,...
716201,"Laboratoire Cardioprotection, Remodelage et Th...",grid.7252.2
716202,"Division of Nephrology, Escola Paulista de Med...",grid.411249.b
716203,Center for Fetal Programming,grid.6203.7
716204,"Jiangsu Kanion Pharmarceutical, Ltd.",grid.452789.5


In [ ]:
###################################################
 ## Quick metrics:
##################################################
###############
 ### Solr institution metrics
###############
  # 1,021,536 out of 1,189,751 total institutions were matched (86%).
  # 521,394 out of 661,753 unique institutions were matched (79%).
###############
 ### EM institution metrics
###############
  # 2,073,332 out of 4,112,509 total institutions were matched (50%).
  # 232,491 out of 500,908 unique institutions were matched (46%).



In [ ]:
##################################################
 ## Export/download the data
##################################################
###############
 ### Download main_insts
###############
from google.colab import files
main_insts.to_csv('main_insts.csv')
files.download('main_insts.csv')

###############
 ### Download em_insts
###############
#em_insts.to_csv('em_insts.csv')
#files.download('em_insts.csv')

###############
 ### Download solr_insts
###############
#solr_insts.to_csv('solr_insts.csv')
#files.download('solr_insts.csv')

###############
 ### Download grid_insts (in case you want the inst_connect column already there)
###############
#grid_insts.to_csv('grid_insts.csv')
#files.download('grid_insts.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>